In [1]:
import numpy as np
import nltk
import pandas as pd

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')  
credits=pd.read_csv('tmdb_5000_credits.csv')

In [5]:
movies.head()
#credits.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
movies=movies.merge(credits,on='title')

In [4]:
movies.columns
## genres, id, keywords, overview, title, cast, crew

## not recommending movie based on budget
## almost all movies are in english hance not focus ro recommeding based on original_language
## popularity is a numerical col but we are trying to create tag for each movies hence not considering this numerical cols.
## we dont recommend movies based on production companies instead we do this by actor director etc
## 

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [5]:
movies=movies[['id','genres',  'keywords', 'overview', 'title', 'cast', 'crew']]

## Data preprocessing

In [7]:
movies.head()

,id,genres,keywords,overview,title,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
movies.isnull().sum()

id          0
genres      0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [7]:
movies.dropna(inplace=True) ## it drops entire row if  there is any missing value in any of the field

In [8]:
movies.duplicated().sum()

0

In [9]:
import json

In [10]:
def get_genres(x):
    lst=json.loads(x)
    name=[]
    for j in range(len(lst)):
        name.append(lst[j]['name'])
    return name

In [11]:
get_genres(movies['genres'][0])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [12]:
movies['genres']=movies['genres'].apply(lambda x: get_genres(x))

In [14]:
#movies.drop(columns='genres',axis=1,inplace=True)

In [11]:
movies.head()


,id,genres,keywords,overview,title,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
movies['keywords']=movies['keywords'].apply(lambda x: get_genres(x))

In [14]:
movies['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

In [15]:
def get_cast_name(x):
    lst=json.loads(x)
    name=[]
    c=0
    for j in range(len(lst)):
        if c!=3:
            name.append(lst[j]['name'])
            c+=1
    return name

In [16]:
movies['cast_name']=movies['cast'].apply(lambda x: get_cast_name(x))

In [17]:
movies.drop(columns='cast',axis=1,inplace=True)

In [18]:
movies.head()

,id,genres,keywords,overview,title,crew,cast_name
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[Christian Bale, Michael Caine, Gary Oldman]"
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,"[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [19]:
def get_dir_name(x):
    lst=json.loads(x)
    name=[]
    for j in range(len(lst)):
        if lst[j]['job']=='Director':
            name.append(lst[j]['name'])
    return name

In [20]:
get_dir_name(movies['crew'][0])

['James Cameron']

In [21]:
movies['crew']=movies['crew'].apply(lambda x: get_dir_name(x))

In [22]:
movies.head()

,id,genres,keywords,overview,title,crew,cast_name
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,[Gore Verbinski],"[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,[Sam Mendes],"[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,[Christopher Nolan],"[Christian Bale, Michael Caine, Gary Oldman]"
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,[Andrew Stanton],"[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [23]:
movies['genres']=movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast_name']=movies['cast_name'].apply(lambda x: [i.replace(" ","") for i in x])

In [24]:
movies.head()

,id,genres,keywords,overview,title,crew,cast_name
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","In the 22nd century, a paraplegic Marine is di...",Avatar,[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver]"
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]"
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",A cryptic message from Bond’s past sends him o...,Spectre,[SamMendes],"[DanielCraig, ChristophWaltz, LéaSeydoux]"
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",Following the death of District Attorney Harve...,The Dark Knight Rises,[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman]"
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","John Carter is a war-weary, former military ca...",John Carter,[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton]"


In [25]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [26]:
movies['tags']=movies['genres'] + movies['keywords'] + movies['overview'] + movies['crew'] + movies['cast_name']

In [27]:
movies.head()

,id,genres,keywords,overview,title,crew,cast_name,tags
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver]","[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,[SamMendes],"[DanielCraig, ChristophWaltz, LéaSeydoux]","[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman]","[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton]","[Action, Adventure, ScienceFiction, basedonnov..."


In [28]:
new_df=movies[['id','title','tags']]

In [29]:
new_df['tags']=new_df['tags'].apply(lambda x: ' '.join(x))

c:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## NLP

In [30]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize

In [31]:
## extracting alphanumeric column

new_df['tags']=new_df['tags'].apply(lambda x: ' '.join(re.findall('\w+',x)))

c:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
##  removing punctuation

new_df['tags']=new_df['tags'].apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))

c:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
## case normalization

new_df['tags']=new_df['tags'].str.lower()

c:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
#nltk.download('punkt')

In [35]:
## tokenization

new_df['tags']=new_df['tags'].apply(lambda x: word_tokenize(x))

c:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
## removing stopwords

stopWord = nltk.corpus.stopwords.words('english')

In [37]:
new_df['tags']=new_df['tags'].apply(lambda x: [i for i in x if i not in stopWord])

c:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
## lemmatization or stemming

ps = nltk.PorterStemmer()

In [39]:
new_df['tags']=new_df['tags'].apply(lambda x: ' '.join([ps.stem(i) for i in x]))

c:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
new_df

,id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...
...,...,...,...
4804,9367,El Mariachi,action crime thriller unitedst mexicobarri leg...
4805,72766,Newlyweds,comedi romanc newlyw coupl honeymoon upend arr...
4806,231617,"Signed, Sealed, Delivered",comedi drama romanc tvmovi date loveatfirstsig...
4807,126186,Shanghai Calling,ambiti new york attorney sam sent shanghai ass...


In [ ]:
##  feature extraction: its time to extract feature
## we have some techniques for representing text into vectors. so that machine can understand the corpus very easily
## 1. onehot encoding
## 2. Bag of words(implementation as CountVectorizer in python/TF-IDF method)
## 3. Word2Vec


## these techniques generally leads to creaation of sparse matrix, more computational time, more memory


## calculation of distance between these vectors: To calculate the similarity between the vectpors we will be calculating 
## the similarity scores between the vectors. hence higher the similarity between the vectors more they close are.
## for this we have eucledian distance, manhattan distance and cosine distance or cosine similarity(-1 to 1).

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
cv=CountVectorizer(max_features=10000)
vectors= cv.fit_transform(new_df['tags']).toarray()
similarity= cosine_similarity(vectors)

## even if we dont mention max_feature as 10000 then it is taking 28900 col and movie in the list is almost same

In [106]:
#similarity= cosine_similarity(vectors)

In [46]:
similarity.shape
vectors.shape

(4806, 10000)

In [43]:
def recommended(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list= sorted(list(enumerate(distances)),key= lambda x: x[1],reverse=True)[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [50]:
pd.set_option('display.max_colwidth', None)

In [51]:
new_df[new_df['title']=='The Dark Knight Rises']['tags']

3    action crime drama thriller dccomic crimefight terrorist secretident burglar hostagedrama timebomb gothamc vigilant cover superhero villai tragichero terror destruct catwoman catburglar imax flood criminalunderworld batman follow death district attorney harvey dent batman assum respons dent crime protect late attorney reput subsequ hunt gotham citi polic depart eight year later batman encount mysteri selina kyle villain bane new terrorist leader overwhelm gotham finest dark knight resurfac protect citi brand enemi christophernolan christianbal michaelcain garyoldman
Name: tags, dtype: object

In [48]:
recommended('The Dark Knight Rises')

The Dark Knight
Batman Begins
Batman
Batman
Batman Returns


In [46]:
new_df

,id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...
...,...,...,...
4804,9367,El Mariachi,action crime thriller unitedst mexicobarri leg...
4805,72766,Newlyweds,comedi romanc newlyw coupl honeymoon upend arr...
4806,231617,"Signed, Sealed, Delivered",comedi drama romanc tvmovi date loveatfirstsig...
4807,126186,Shanghai Calling,ambiti new york attorney sam sent shanghai ass...


In [144]:
# movie_index=new_df[new_df['title']=='Avatar'].index[0]
# distance=similarity[movie_index]
# #sorted(distance,reverse=True)
# movie_list= sorted(list(enumerate(distance)),key= lambda x: x[1],reverse=True)[1:6]
# for i in movie_list:
#     print(new_df.iloc[i[0]].title)

Aliens
Falcon Rising
Titan A.E.
Aliens vs Predator: Requiem
Battle: Los Angeles


In [168]:
import pickle

In [169]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [170]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [171]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [39]:
# from sklearn.base import TransformerMixin

# # Define a custom transformer for computing cosine similarity
# class CosineSimilarityTransformer(TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         return cosine_similarity(self.X, self.X)

# # Define the pipeline with CountVectorizer and cosine similarity
# pipeline = Pipeline([
#     ('vect', CountVectorizer()),
#     ('cosine_sim', CosineSimilarityTransformer())  # Custom transformer for cosine similarity
# ])

# # Define the parameter grid for grid search
# param_grid = {
#     'vect__max_features': [1000, 5000, 10000] # Example values for max_features
#     # Other parameters for CountVectorizer can also be included here
#     # Example:
#     # 'vect__ngram_range': [(1, 1), (1, 2)],
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(pipeline, param_grid, cv=5)

# # Fit the grid search to the text data
# grid_search.fit(new_df['tags'])

# # Get the best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)


GridSearchCV needs a metric to optimize. A vectorizer doesn’t have anything like an accuracy, so it doesn’t make sense to
run any kind of hyperparameter optimization on your vectorize method.

The main issue is that it’s impossible to tell whether a change in your vectorizer will lead to higher or lower accuracy in 
your resulting model, unless you keep the model constant.

That said, if you do wish to keep your model constant and optimize your vectorizer in that light, then you can easily create 
a customer function that takes in kwargs that will be passed to the vectorizer, and return appropriate accuracy metrics.

Alternately you can create a function that takes a vectorizer as a kwarg, then generate several samples ahead of time and then
simply add them to the params argument you’re passing to GridSearchCV

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('vect', CountVectorizer()),
                ('cosine_sim',
                 <__main__.CosineSimilarityTransformer object at 0x00000255A14C1AC8>)]) does not.

In [55]:
recommended('Spectre')

Quantum of Solace
Never Say Never Again
Skyfall
From Russia with Love
Thunderball


In [36]:
recommended('A Walk to Remember')

The Life of David Gale
The Green Mile
Moonlight Mile
To Kill a Mockingbird
Find Me Guilty


In [57]:
recommended2('Basic Instinct 2')

Blood Ties
City By The Sea
Broken Horses
Regression
The Corruptor


In [54]:
new_df['title']

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4804                                 El Mariachi
4805                                   Newlyweds
4806                   Signed, Sealed, Delivered
4807                            Shanghai Calling
4808                           My Date with Drew
Name: title, Length: 4806, dtype: object